In [ ]:
!pip install pycaret

In [ ]:
import pandas as pd

In [ ]:
dataset=pd.read_csv('../input/bike-sharing/day.csv')
dataset.head()

In [ ]:
dataset['dteday'] =  pd.to_datetime(dataset['dteday'], infer_datetime_format=True)
dataset.head()

In [ ]:
dataset.info()

In [ ]:
data = dataset.sample(frac=0.8, random_state=42)
data_unseen = dataset.drop(data.index)
data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)
print('Data for Modeling: ' + str(data.shape))

In [ ]:
print('Unseen Data For Predictions ' + str(data_unseen.shape))

In [ ]:
from pycaret.regression import *

In [ ]:
bike= setup(data = data, target = 'cnt', session_id=1,
                  normalize = True, transformation = True, transform_target = True, 
                  combine_rare_levels = True, rare_level_threshold = 0.05,
                  remove_multicollinearity = True, multicollinearity_threshold = 0.98,
                  log_experiment = True, experiment_name = 'bike')

In [ ]:
best = compare_models()

In [ ]:
catboost = create_model('catboost')

In [ ]:
print(catboost)

In [ ]:
tuned_catboost = tune_model(catboost)

In [ ]:
plot_model(tuned_catboost)

In [ ]:
plot_model(tuned_catboost, plot = 'error')

In [ ]:
plot_model(tuned_catboost, plot='feature')

In [ ]:
evaluate_model(tuned_catboost)

In [ ]:
predict_model(tuned_catboost)

In [ ]:
final_catboost = finalize_model(tuned_catboost)
final_catboost

In [ ]:
predict_model(final_catboost)

In [ ]:
unseen_predictions = predict_model(final_catboost, data=data_unseen)
unseen_predictions.head()

In [ ]:
from pycaret.utils import check_metric
check_metric(unseen_predictions.cnt, unseen_predictions.Label, 'MAE')

In [ ]:
save_model(final_catboost,'./model')

In [ ]:
saved_final_catboost = load_model('./model')

In [ ]:
new_prediction = predict_model(saved_final_catboost, data=data_unseen)

In [ ]:
new_prediction.head()

In [ ]:
from pycaret.utils import check_metric
check_metric(new_prediction.cnt, new_prediction.Label, 'MAE')